# Vehicle Classification Report on Cropped dataset

In [1]:
import os

from sklearn.metrics import accuracy_score, classification_report

from utils import utils
from models import resnet_50

In this first section are shown some results obtained training our model with cropped data set, applying removing background.

Weights were initialized with our previous fine tunning results.

## TEST #1

For this test, all previous parameters from last train were kept.Only a learning rate variation was chosen to be less drastin (from 0.7 to 0.5), increased the patience and cooldown.

![exp_3 training](resources/exp_3_train.PNG "Training results from experiment 3")

A validation accuracy of 0.68 was obtained at epoch 97.

![exp_3 acc](resources/exp_3_acc.PNG "Accuracy vs epoch - experiment 3")
![exp_3 loss](resources/exp_3_loss.PNG "Loss vs epoch - experiment 3")

There is a better accuracy than training with original imeages and also less overfiting

### Configuration file

In [2]:
CONFIG_YML = "../experiments/exp_3/config.yml"
WEIGHTS = "../experiments/exp_3/model.97-0.6800.h5"
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '/home/app/src/data/car_ims_v2/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': 'imagenet',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.5,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_rotation': {'factor': 0.3},
   'random_zoom': {'height_factor': 0.2, 'width_factor': 0.2}},
  'regularizers': {'kernel_regularizer': {'L2': {'l2': 0.001}},
   'activity_regularizer': {'L2': {'l2': 0.001}}},
  'output_regularizer': {'kernel_regularizer': {'L2': {'l2': 0.001}}},
  'n_dense_layers': 2,
  'n_unfreeze_layers': 8,
  'trainable': True},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.0001}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 100,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experiments/exp_3/model.{epoch:02d}-{val_accuracy

In [3]:
# Infer the class names and also load the corresponding testing dataset.

MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

### Loading the model


In [10]:
cnn_model = resnet_50.create_model(weights=WEIGHTS, trainable=False)
print(cnn_model.summary())

2022-11-22 14:19:24.848177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 14:19:24.857960: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 14:19:24.858528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 14:19:24.860091: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

### Get predictions from testing dataset

In [11]:

#     Get model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


2022-11-22 14:19:39.522235: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-11-22 14:19:39.988702: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-22 14:19:39.989337: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-22 14:19:39.989408: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-22 14:19:39.990060: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-22 14:19:39.990210: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-11-22 14:19:41.454154: W tensorflow/core/common_runtime/bf

In [12]:
print(classification_report(y_true=labels, y_pred=predictions))

                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.79      0.93      0.85        44
                             Acura Integra Type R 2001       0.75      0.48      0.58        44
                                   Acura RL Sedan 2012       0.40      0.12      0.19        32
                                   Acura TL Sedan 2012       0.62      0.72      0.67        43
                                  Acura TL Type-S 2008       0.62      0.60      0.61        42
                                  Acura TSX Sedan 2012       0.60      0.30      0.40        40
                              Acura ZDX Hatchback 2012       0.56      0.69      0.62        39
              Aston Martin V8 Vantage Convertible 2012       0.40      0.42      0.41        45
                    Aston Martin V8 Vantage Coupe 2012       0.37      0.27      0.31        41
                  Aston Martin Virage C

In [13]:
acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.6227!


## Test 2

For this test a fine-tuning was performed starting from the previous weights

A maximum validation accuracy of 0.8446 was obtained at epoch 52; and 0.8329 at the last epoch 75

![exp_4 acc](resources/exp_4_acc.PNG "Accuracy vs epoch - experiment 4")
![exp_3 loss](resources/exp_3_loss.PNG "Loss vs epoch - experiment 3")

Model is overfitting again (train accuracy very close to 1),  similar to the model without cropped images.

In [12]:
CONFIG_YML_RETRAIN = "../experiments/exp_4/config.yml"
WEIGHTS_RETRAIN = "../experiments/exp_4/model.75-0.8329.h5"
config_retrain = utils.load_config(CONFIG_YML_RETRAIN)
config_retrain

{'seed': 123,
 'data': {'directory': '/home/app/src/data/car_ims_v2/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': '/home/app/src/experiments/exp_3/model.97-0.6800.h5',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.5,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_rotation': {'factor': 0.3},
   'random_zoom': {'height_factor': 0.2, 'width_factor': 0.2}},
  'regularizers': {'kernel_regularizer': {'L2': {'l2': 0.001}},
   'activity_regularizer': {'L2': {'l2': 0.001}}},
  'output_regularizer': {'kernel_regularizer': {'L2': {'l2': 0.001}}},
  'n_dense_layers': 2,
  'n_unfreeze_layers': 8,
  'trainable': True},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.0001}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 100,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experim

In [13]:
cnn_model_retrain = resnet_50.create_model(weights=WEIGHTS_RETRAIN, trainable=False)
print(cnn_model_retrain.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

In [14]:
#     Get model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions_retrain, labels_retrain = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model_retrain, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions_retrain) != len(labels_retrain):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions_retrain[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels_retrain[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


In [15]:
print(classification_report(y_true=labels_retrain, y_pred=predictions_retrain))

                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.71      0.89      0.79        44
                             Acura Integra Type R 2001       0.93      0.91      0.92        44
                                   Acura RL Sedan 2012       0.81      0.69      0.75        32
                                   Acura TL Sedan 2012       0.80      0.95      0.87        43
                                  Acura TL Type-S 2008       0.93      0.98      0.95        42
                                  Acura TSX Sedan 2012       0.84      0.68      0.75        40
                              Acura ZDX Hatchback 2012       0.88      0.90      0.89        39
              Aston Martin V8 Vantage Convertible 2012       0.54      0.82      0.65        45
                    Aston Martin V8 Vantage Coupe 2012       0.77      0.56      0.65        41
                  Aston Martin Virage C

In [16]:
acc = accuracy_score(y_true=labels_retrain, y_pred=predictions_retrain)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.8413!


## Conclussion

- As the dataset is well balanced, accuracy give as a good estimation of our model performance.
- Removing the background from the images using detectron2 considetable improved the accuracy of our models. 
- Reducing overfiting was a very difficult task, increasing droup rate and regularization did not help to prevent it. Increasing them a lot made the model to not evenlearnfrom the dataset.
- Even though we had much higher accuracy in training with respect to validation (overfitting), our models performed very similar in test set in comparisson of validation, which indicates robustness.
- Experimenting with sgd optimizer instead of adam does not seem to have considerable impact. Playing around with different parameters and hhyperparametes along with sgdoptimizer are needed to have  fair comparisson.
- Adding more layers on top of renet_50 model shows a high improvement with respect to adding only the output layer in the first train (starting from pre trainned imagenet weight). In the fine tunning step, similar results are obtained with more dense layers or without them.
- Given the small dataset and the reduced amount of resources (hardware and time) quite good results were obtained.